In [ ]:
#pip install opencv-python

In [1]:
import cv2
import numpy as np
cover_path='.\Test-Videos-Image\q2-cover_vid.mp4'
secret_path='.\Test-Videos-Image\q2-secret_vid.mp4'
def VideoToTensor(path):
    cap = cv2.VideoCapture(path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  
    
        frame_array = np.array(frame)
        frames.append(frame_array)

    cap.release()
    tensor_video= np.array(frames) 
    return tensor_video


In [2]:
cover_video=VideoToTensor(cover_path)
secret_video=VideoToTensor(secret_path)

In [3]:
C = np.transpose(cover_video, (1, 2, 0, 3))
S = np.transpose(secret_video, (1, 2, 0, 3))

In [4]:
C.shape

(144, 144, 20, 3)

In [5]:
S.shape

(144, 144, 20, 3)

In [6]:
def mDomain(A):
    Ma,Na,Pa,Qa=A.shape
    Am=np.zeros((Ma,Na,Pa,Qa))
    M3=np.diag(range(1,Pa+1))
    M3_inv = np.linalg.inv(M3)
    for i in range(Qa):
        Am[:,:,:,i]=np.einsum("lk,ijk->ijl",M3,A[:,:,:,i])
    M4=np.diag(range(1,Qa+1))
    M4_inv = np.linalg.inv(M4)
    Amm=np.einsum("lk,ijqk->ijql",M4,Am)
    return Amm
    
def mTotDomain(A):
    Ma,Na,Pa,Qa=A.shape
    Amm=np.zeros((Ma,Na,Pa,Qa))
    M4=np.diag(range(1,Qa+1))
    M4_inv = np.linalg.inv(M4)
    Am=np.einsum("lk,ijqk->ijql",M4_inv,A)
    M3=np.diag(range(1,Pa+1))
    M3_inv = np.linalg.inv(M3)
    for i in range(Qa):
        Amm[:,:,:,i]=np.einsum("lk,ijk->ijl",M3_inv,Am[:,:,:,i])
    return Amm

def m4Product(A,B):
    Ma,Na,Pa,Qa=A.shape
    Mb,Nb,Pb,Qb=B.shape
    Am=mDomain(A)
    Bm=mDomain(B)
    ABm=np.zeros((Ma,Nb,Pa,Qa))
    for i in range(Pa):
        for j in range(Qa):
            ABm[:,:,i,j]=np.dot(Am[:,:,i,j],Bm[:,:,i,j])
    AB=mTotDomain(ABm)
    return AB

In [7]:
import numpy as np
from scipy.linalg import qr
def QRdecomposition(A):
    m,n,p,q=A.shape
    
    Am=mDomain(A)
    k=p*q
    
    Q_prm=np.zeros((m,n,p,q))
    R_prm=np.zeros((m,n,p,q))
    for i in range(p):
        for j in range(q):
            Q_t,R_t=qr(Am[:,:,i,j])
            Q_prm[:,:,i,j]=Q_t
            R_prm[:,:,i,j]=R_t
            
    Q=mTotDomain(Q_prm)
    R=mTotDomain(R_prm)
    
    return Q,R
        
        

In [8]:
Qc,Rc=QRdecomposition(C)

In [9]:
Qs,Rs=QRdecomposition(S)

In [10]:
Rt=Rc+(0.1*Rs)


In [11]:
T=m4Product(Qc,Rt)

In [12]:
T.shape

(144, 144, 20, 3)

In [13]:
Transfer_video = np.transpose(T, (2, 0, 1, 3))    

In [14]:
Transfer_video.shape

(20, 144, 144, 3)

In [15]:
Qt,Rtt=QRdecomposition(T)

In [16]:
def save_video(tensor, filename="output.mp4", fps=30):
    
    T, H, W, C = tensor.shape
    assert C in [1, 3], "Only grayscale (C=1) or RGB (C=3) supported."

   
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    out = cv2.VideoWriter(filename, fourcc, fps, (W, H))

    for t in range(T):
        frame = tensor[t] 
        frame = (frame).astype(np.uint8) 
        if C == 3:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  
        out.write(frame)

    out.release()
    print(f"Video saved as {filename}")


save_video(Transfer_video, "Output\Q2\CoverVideoTranfered-MProduct-Output.mp4", fps=5)

Video saved as Output\Q2\CoverVideoTranfered-MProduct-Output.mp4


In [17]:
Ro=(Rtt-Rc)/0.1

In [18]:
O=m4Product(Qs,Ro)

In [19]:
O.shape

(144, 144, 20, 3)

In [20]:
Org_video = np.transpose(O, (2, 0, 1, 3)) 
Org_video.shape

(20, 144, 144, 3)

In [21]:
save_video(Org_video, "Output\Q2\OriginalVideoTranfered-MProduct-Output.mp4", fps=5)

Video saved as Output\Q2\OriginalVideoTranfered-MProduct-Output.mp4
